-CS24MTECH11023 | CHALASANI VINEETH

In [1]:
import csv
import random
import numpy as np
from scipy.linalg import null_space
import numpy as np
np.random.seed(0)

In [2]:
def load_data(filename):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)
    z = np.array([float(val) for val in data[0][:-1]])
    c = np.array([float(val) for val in data[1][:-1]])
    b = np.array([float(row[-1]) for row in data[2:]])
    A = np.array([[float(val) for val in row[:-1]] for row in data[2:]])
    m, n = len(b), len(c)
    return A, z, b, c, m, n

In [3]:
def calculate_product_az(_A, _z):
  return np.dot(_A, _z)

def get_tight_mask(_product, _b, epi=1e-8):
  return np.abs(_product - _b) < epi

def find_tight_rows(_A, _z, _b, epi=1e-8):
  product = calculate_product_az(_A, _z)
  tight_mask = get_tight_mask(product, _b, epi)
  A1 = _A[tight_mask]
  A2 = _A[~tight_mask]
  return tight_mask, A1, A2

In [4]:
def get_directions(_A1):
  try:
    A_inv = np.linalg.inv(_A1)
    neg_A_inv = -1 * A_inv
    return neg_A_inv
  except np.linalg.LinAlgError:
    print("Matrix is singular. Cannot compute the inverse.")
    return None

In [5]:
def is_degenerate(tight):
  return tight.shape[0] > tight.shape[1]

In [6]:
def make_non_degenerate(vector_b_original, epi, reduction_factor=0.5):
  epi = epi*reduction_factor
  new_b  = np.array([vector_b_original[i]+epi**(i+1) for i in range(len(vector_b_original))])
  return new_b, epi

In [7]:
def get_modified_LP(_A, _b, _c):
  min_b = min(_b)
  initial_point = create_initial_point(_A.shape, min_b)

  modified_A, modified_b, modified_c = _A, _b, _c

  if initial_point.shape[0] != _c.shape[0]:
    modified_A, modified_b, modified_c = modify_matrices_for_LP(_A, _b, _c, min_b)

  return modified_A, modified_b, np.hstack(modified_c), np.hstack(initial_point)

In [8]:
def create_initial_point(A_shape, min_b):
  initial_point = np.zeros((A_shape[1] + 1, 1))
  initial_point[-1] = min_b
  return initial_point

In [9]:
def modify_matrices_for_LP(_A, _b, _c, min_b):
  rows, cols = _A.shape
  modified_A = np.append(np.append(_A, np.zeros((1, cols)), axis=0), np.ones((rows + 1, 1)), axis=1)
  modified_A[-1][-1] = -1
  modified_b = np.append(_b, [abs(min_b)], axis=0)
  modified_c = np.zeros((cols + 1, 1))
  modified_c[-1] = 1
  return modified_A, modified_b, modified_c

In [10]:
def initialize_feasible_to_vertex(_vector_z, _vector_c):
  track_cost = []
  track_z = []
  track_cost.append(np.dot(_vector_c, _vector_z))
  track_z.append(_vector_z)
  z_old = _vector_z
  iteration = 0
  print_interval = 1
  return track_cost, track_z, z_old, iteration, print_interval

In [11]:
def calculate_step_magnitude(_vector_b, mask, untight_rows, z_old, u):
  alphas = [(_b_i - np.dot(a2_i, z_old)) / np.dot(a2_i, u) for _b_i, a2_i in zip(_vector_b[~mask], untight_rows)]
  alphas = [alpha for alpha in alphas if alpha > 0]
  positive_alphas = []
  epi = 1e-10
  for i in alphas:
    if not(i == np.inf or abs(i) < epi):
      positive_alphas.append(i)
  return positive_alphas

In [12]:
def get_direction(_matrix_A, _vector_z, _vector_b, tight_rows):
  if len(tight_rows) == 0:
    u = np.random.rand(_matrix_A.shape[-1])
  else:
    null_space_matrix = null_space(tight_rows)
    u = null_space_matrix[:, 0]
  return u

In [13]:
def compute_constraint_rank(active_constraint_matrix):
    if len(active_constraint_matrix) == 0:
        return 0
    else:
        return np.linalg.matrix_rank(active_constraint_matrix)

def setup_vertex_search_variables(starting_point, objective_coefficients):
    cost_history, point_sequence, current_point, iteration_counter, progress_interval = initialize_feasible_to_vertex(starting_point, objective_coefficients)
    return cost_history, point_sequence, current_point, iteration_counter, progress_interval

def manage_iteration_progress(iteration_counter, progress_interval, constraint_rank):
    if iteration_counter % progress_interval == 0:
        print(f"Iteration: {iteration_counter} - Rank: {constraint_rank}")
        if iteration_counter > 300:
            progress_interval = 1000
        elif iteration_counter > 10000:
            progress_interval = 10000
    return progress_interval

def calculate_movement_direction(constraint_matrix, current_point, constraint_bounds, active_constraints):
    return get_direction(constraint_matrix, current_point, constraint_bounds, active_constraints)

def determine_valid_step_size(constraint_bounds, constraint_mask, inactive_constraints, current_point, movement_vector):
    valid_step_sizes = calculate_step_magnitude(constraint_bounds, constraint_mask, inactive_constraints, current_point, movement_vector)
    
    while len(valid_step_sizes) == 0:
        movement_vector = -1 * movement_vector
        valid_step_sizes = calculate_step_magnitude(constraint_bounds, constraint_mask, inactive_constraints, current_point, movement_vector)
        break
    
    return min(valid_step_sizes), movement_vector

def update_search_state(constraint_matrix, new_point, constraint_bounds, objective_coefficients, cost_history, point_sequence):
    constraint_mask, active_constraints, inactive_constraints = find_tight_rows(constraint_matrix, new_point, constraint_bounds)
    constraint_rank = compute_constraint_rank(active_constraints)
    
    cost_history.append(np.dot(objective_coefficients, new_point))
    point_sequence.append(new_point)
    
    return constraint_mask, active_constraints, inactive_constraints, constraint_rank

def execute_feasible_point_to_vertex_search(constraint_matrix, constraint_bounds, starting_point, objective_coefficients, target_dimension):
    cost_history, point_sequence, current_point, iteration_counter, progress_interval = setup_vertex_search_variables(starting_point, objective_coefficients)
    
    constraint_mask, active_constraints, inactive_constraints = find_tight_rows(constraint_matrix, starting_point, constraint_bounds)
    constraint_rank = compute_constraint_rank(active_constraints)
    
    if constraint_rank == target_dimension:
        return current_point, cost_history, point_sequence
    else:
        print("Feasible point is not a vertex. Searching for a vertex...")
    
    while constraint_rank != target_dimension:
        iteration_counter += 1
        progress_interval = manage_iteration_progress(iteration_counter, progress_interval, constraint_rank)
        
        movement_vector = calculate_movement_direction(constraint_matrix, current_point, constraint_bounds, active_constraints)
        step_size, adjusted_movement = determine_valid_step_size(constraint_bounds, constraint_mask, inactive_constraints, current_point, movement_vector)
        
        new_point = current_point + step_size * adjusted_movement
        current_point = new_point
        
        constraint_mask, active_constraints, inactive_constraints, constraint_rank = update_search_state(constraint_matrix, new_point, constraint_bounds, objective_coefficients, cost_history, point_sequence)
    
    if not(is_degenerate(active_constraints)):
        return (new_point, cost_history, point_sequence)
    else:
        return (None,)

def feasible_to_vertex_assign3(constraint_matrix, constraint_bounds, starting_point, objective_coefficients, target_dimension):
    return execute_feasible_point_to_vertex_search(constraint_matrix, constraint_bounds, starting_point, objective_coefficients, target_dimension)

In [14]:
def initialize_optimization_tracking(starting_vertex, objective_coefficients):
    cost_history = []
    vertex_sequence = []
    current_vertex = starting_vertex
    next_vertex = current_vertex
    cost_history.append(np.dot(objective_coefficients, current_vertex))
    vertex_sequence.append(current_vertex)
    iteration_count = 0
    display_frequency = 1
    return cost_history, vertex_sequence, current_vertex, next_vertex, iteration_count, display_frequency

def compute_feasible_movement_directions(constraint_matrix, current_vertex, constraint_bounds, objective_coefficients):
    constraint_mask, active_constraints, inactive_constraints = find_tight_rows(constraint_matrix, current_vertex, constraint_bounds)
    
    if is_degenerate(active_constraints):
        return None, None, None, True
    
    movement_directions = get_directions(active_constraints).T
    beneficial_directions = []
    
    for direction_vector in movement_directions:
        if np.dot(direction_vector, objective_coefficients) > 0:
            beneficial_directions.append(direction_vector)
    
    return constraint_mask, inactive_constraints, beneficial_directions, False

def calculate_maximum_step_length(constraint_bounds, constraint_mask, inactive_constraints, current_vertex, movement_direction):
    step_candidates = [(bound_value - np.dot(constraint_row, current_vertex)) / np.dot(constraint_row, movement_direction) 
                      for bound_value, constraint_row in zip(constraint_bounds[~constraint_mask], inactive_constraints)]
    
    positive_steps = [step for step in step_candidates if step > 0]
    
    valid_steps = []
    tolerance_threshold = 1e-10
    
    for step_value in positive_steps:
        if not(step_value == np.inf or abs(step_value) < tolerance_threshold):
            valid_steps.append(step_value)
    
    return valid_steps

def execute_vertex_optimization_algorithm(constraint_matrix, constraint_bounds, starting_vertex, objective_coefficients, problem_dimensions):
    cost_history, vertex_sequence, current_vertex, next_vertex, iteration_count, display_frequency = initialize_optimization_tracking(starting_vertex, objective_coefficients)
    
    while True:
        iteration_count += 1
        if iteration_count % display_frequency == 0:
            print(f"Iteration: {iteration_count}")

        constraint_mask, inactive_constraints, beneficial_directions, degeneracy_detected = compute_feasible_movement_directions(constraint_matrix, current_vertex, constraint_bounds, objective_coefficients)
        
        if degeneracy_detected:
            return (None,)

        if not beneficial_directions or iteration_count > 10:
            return next_vertex, cost_history, vertex_sequence

        optimal_direction = beneficial_directions[0]

        valid_steps = calculate_maximum_step_length(constraint_bounds, constraint_mask, inactive_constraints, current_vertex, optimal_direction)

        if len(valid_steps) == 0:
            print(f"The problem is unbounded. Can't find a optimal solution!")
            return None, cost_history, vertex_sequence

        maximum_step = min(valid_steps)
        next_vertex = current_vertex + maximum_step * optimal_direction
        current_vertex = next_vertex

        cost_history.append(np.dot(objective_coefficients, next_vertex))
        vertex_sequence.append(next_vertex)

def vertex_to_vertex_assign3(constraint_matrix, constraint_bounds, starting_vertex, objective_coefficients, problem_dimensions):
    return execute_vertex_optimization_algorithm(constraint_matrix, constraint_bounds, starting_vertex, objective_coefficients, problem_dimensions)

In [15]:
def validate_origin_feasibility(constraint_bounds):
    return np.all(constraint_bounds >= 0)

def configure_auxiliary_linear_program(constraint_matrix, constraint_bounds, objective_coeffs):
    auxiliary_matrix, auxiliary_bounds, auxiliary_objective, initial_auxiliary_point = get_modified_LP(constraint_matrix, constraint_bounds, objective_coeffs)
    num_auxiliary_rows, num_auxiliary_cols = len(auxiliary_bounds), len(auxiliary_objective)
    
    print("Initial Feasible Point (z):", initial_auxiliary_point)
    print("Cost Vector (c):", auxiliary_objective)
    print("Constraint Vector (b):", auxiliary_bounds)
    print("Matrix A:")
    print(auxiliary_matrix)
    print(f"Rows: {num_auxiliary_rows} | Columns: {num_auxiliary_cols}")
    
    return auxiliary_matrix, auxiliary_bounds, auxiliary_objective, initial_auxiliary_point, num_auxiliary_cols

def execute_auxiliary_optimization_routine(constraint_matrix, constraint_bounds, starting_point, objective_vector, problem_dimension):
    tolerance_epsilon = 0.1
    degeneracy_attempt = 0
    working_bounds = constraint_bounds
    
    while True:
        if degeneracy_attempt > 0:
            print(f"Degeneracy detected. Attempting to handle it. Attempt - {degeneracy_attempt}")
            working_bounds, tolerance_epsilon = make_non_degenerate(constraint_bounds, tolerance_epsilon)

        phase1_results = feasible_to_vertex_assign3(constraint_matrix, working_bounds, starting_point, objective_vector, problem_dimension)
        if len(phase1_results) == 1:
            degeneracy_attempt += 1
            continue

        print("Reached the initial vertex from feasible point!")
        vertex_point, phase1_costs, phase1_vertices = phase1_results

        print("Searching for optimal vertex...")
        phase2_results = vertex_to_vertex_assign3(constraint_matrix, working_bounds, vertex_point, objective_vector, problem_dimension)
        if len(phase2_results) == 1:
            degeneracy_attempt += 1
            continue

        optimal_solution, phase2_costs, phase2_vertices = phase2_results
        
        if np.all(optimal_solution == None):
            solution_status = None
            print("The problem is unbounded!")
        else:
            solution_status = -1
            print("Reached the optimal vertex!")
        break
    
    return optimal_solution, solution_status, vertex_point, phase1_costs, phase1_vertices, phase2_costs, phase2_vertices

def handle_unbounded_auxiliary_case(original_feasible_point):
    print("The modified LP is unbounded! Therefore the given problem is unbounded!")
    print("The following is the initial feasible point for the original problem:")
    print(original_feasible_point)

def determine_original_feasible_solution(constraint_matrix, constraint_bounds, objective_coeffs):
    original_feasible_point = None
    
    if validate_origin_feasibility(constraint_bounds):
        original_feasible_point = np.zeros(objective_coeffs.shape)
        print("Using 0 origin as the initial feasible point!")
        return (original_feasible_point,)
    else:
        print("Solving the modified LP!")
        auxiliary_matrix, auxiliary_bounds, auxiliary_objective, initial_auxiliary_point, auxiliary_dimension = configure_auxiliary_linear_program(constraint_matrix, constraint_bounds, objective_coeffs)
        
        optimal_solution, solution_status, vertex_point, phase1_costs, phase1_vertices, phase2_costs, phase2_vertices = execute_auxiliary_optimization_routine(auxiliary_matrix, auxiliary_bounds, initial_auxiliary_point, auxiliary_objective, auxiliary_dimension)
        
        if solution_status == None:
            handle_unbounded_auxiliary_case(original_feasible_point)
        
        return (optimal_solution, solution_status, vertex_point, phase1_costs, phase1_vertices, optimal_solution, phase2_costs, phase2_vertices)

In [16]:
A, z, b, c, m, n = load_data('testcase_9.csv')

print("Initial Feasible Point (z):", z)
print("Cost Vector (c):", c)
print("Constraint Vector (b):", b)
print("Matrix A:")
print(A)
print(f"Rows: {m} | Columns: {n}")

Initial Feasible Point (z): [3. 9.]
Cost Vector (c): [ 0. -1.]
Constraint Vector (b): [0. 8. 4.]
Matrix A:
[[-1.  0.]
 [ 1.  4.]
 [ 1.  2.]]
Rows: 3 | Columns: 2


OUTPUT

In [17]:
outputs = determine_original_feasible_solution(A, b, c)
z = None
if len(outputs) > 1:
  z_optimal, modified_z_optimal, z_new, feas2vert_z_all_cost, feas2vert_z_all, z_optimal, vert2vert_z_all_cost, vert2vert_z_all = outputs
  if modified_z_optimal != None:
    z = z_optimal[:len(c)]
else:
  z = outputs[0]
  modified_z_optimal = -1

Using 0 origin as the initial feasible point!


In [18]:
if modified_z_optimal == None:

    print("The problem is unbounded!")

else:
  matrix_A = A
  vector_b_original = b
  vector_z = z
  vector_c = c
  dimension_n = n

  epsilon = 0.1
  attempt = 0
  vector_b = vector_b_original
  attempt = 0
  while True:
    if attempt > 0:

      print(f"Degeneracy detected. Attempting to handle it. Attempt - {attempt}")

      vector_b, epsilon = make_non_degenerate(vector_b_original, epsilon)

    outputs1 = feasible_to_vertex_assign3(matrix_A, vector_b, vector_z, vector_c, dimension_n)
    if len(outputs1) == 1:
      attempt+=1
      continue


    print("Reached the initial vertex from feasible point!")

    z_new, feas2vert_z_all_cost, feas2vert_z_all = outputs1

    print("Searching for optimal vertex...")
    outputs2 = vertex_to_vertex_assign3(matrix_A, vector_b, z_new, vector_c, dimension_n)
    if len(outputs2) == 1:
      attempt+=1
      continue

    z_optimal, vert2vert_z_all_cost, vert2vert_z_all = outputs2
    if np.all(z_optimal == None):

      print("The problem is unbounded!")

    else:

      print("Reached the optimal vertex!")

    break

Feasible point is not a vertex. Searching for a vertex...
Iteration: 1 - Rank: 1
Degeneracy detected. Attempting to handle it. Attempt - 1
Feasible point is not a vertex. Searching for a vertex...
Iteration: 1 - Rank: 0
Iteration: 2 - Rank: 1
Reached the initial vertex from feasible point!
Searching for optimal vertex...
Iteration: 1
The problem is unbounded. Can't find a optimal solution!
The problem is unbounded!


In [19]:
if modified_z_optimal == None:
    print("The problem is unbounded! \nSo we are displaying the values for the modified LP which was used to find the initial feasible point!")
    print("\n--- Feasible point to vertex ---")
    print("Sequence of vertices visited and objective function values:")
    for i, (vertex, cost) in enumerate(zip(feas2vert_z_all, feas2vert_z_all_cost)):
        print(f"Vertex {i+1}: {vertex}, Objective Value: {cost}")

    print("\n--- Vertex to Optimal vertex ---")
    print("Sequence of vertices visited and objective function values:")
    for i, (vertex, cost) in enumerate(zip(vert2vert_z_all, vert2vert_z_all_cost)):
        print(f"Vertex {i+1}: {vertex}, Objective Value: {cost}")

    if np.all(z_optimal == None):
      print("\nThe problem is unbounded!")
    else:
      print("\nOptimal vertex")
      print(z_optimal)

else:
  print("\n--- Feasible point to vertex ---")
  print("Sequence of vertices visited and objective function values:")
  for i, (vertex, cost) in enumerate(zip(feas2vert_z_all, feas2vert_z_all_cost)):
      print(f"Vertex {i+1}: {vertex}, Objective Value: {cost}")

  print("\n--- Vertex to Optimal vertex ---")
  print("Sequence of vertices visited and objective function values:")
  for i, (vertex, cost) in enumerate(zip(vert2vert_z_all, vert2vert_z_all_cost)):
      print(f"Vertex {i+1}: {vertex}, Objective Value: {cost}")

  if np.all(z_optimal == None):
    print("\nThe problem is unbounded!")
  else:
    print("\nOptimal vertex")
    print(z_optimal)


--- Feasible point to vertex ---
Sequence of vertices visited and objective function values:
Vertex 1: [0. 0.], Objective Value: 0.0
Vertex 2: [1.10920131 1.44546185], Objective Value: -1.4454618460765258
Vertex 3: [-0.00225    2.0011875], Objective Value: -2.0011875

--- Vertex to Optimal vertex ---
Sequence of vertices visited and objective function values:
Vertex 1: [-0.00225    2.0011875], Objective Value: -2.0011875

The problem is unbounded!
